# Data Load

In [1]:
import config
from pshmodule.utils import filemanager as fm

In [2]:
df = fm.load(config.temp_data)

extension : .pickle
Loaded 9150 records from /Volumes/GoogleDrive/내 드라이브/MemeProject/data/temp_for_doc2vec.pickle


In [3]:
df.head()

1,content,morphs
0,나 중간고사 반에서 1등했어,"[나, 중간고사, 반, 에서, 1등, 했어]"
1,중간고사 점수 내가 반에서 제일 잘 받음,"[중간고사, 점수, 내, 가, 반, 에서, 제일, 잘, 받음]"
2,나 반에서 중간 성적 제일 좋아,"[나, 반, 에서, 중간, 성적, 제일, 좋아]"
3,우리 반에서 내가 시험 제일 잘 봤다,"[우리, 반, 에서, 내, 가, 시험, 제일, 잘, 봤다]"
4,중간고사 반 1등 먹음,"[중간고사, 반, 1등, 먹음]"


# 

# doc2vec

In [4]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [5]:
tagged_data = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(df.morphs)]

In [6]:
print(tagged_data[:5])

[TaggedDocument(words=['나', '중간고사', '반', '에서', '1등', '했어'], tags=['0']), TaggedDocument(words=['중간고사', '점수', '내', '가', '반', '에서', '제일', '잘', '받음'], tags=['1']), TaggedDocument(words=['나', '반', '에서', '중간', '성적', '제일', '좋아'], tags=['2']), TaggedDocument(words=['우리', '반', '에서', '내', '가', '시험', '제일', '잘', '봤다'], tags=['3']), TaggedDocument(words=['중간고사', '반', '1등', '먹음'], tags=['4'])]


In [7]:
model = Doc2Vec(tagged_data, vector_size=1024, window=3, epochs=100, min_count=0, workers=4)

### model save

In [8]:
model.save(config.doc2vec)
print("Model Saved")

Model Saved


# 